In [1]:
## import libraries
import sys
import glob
import re

import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec

# cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from plotter import draw_basemap, plot_terrain
from utils import roundPartial, select_months_ds
import customcmaps as ccmap

pd.options.display.float_format = "{:,.2f}".format # makes it so pandas tables display only first two decimals

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/' 
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## Load Basin watershed file
fp = path_to_data + 'downloads/UpperCo/Upper_Colorado_River_Basin_Boundary.shp'
basin = gpd.read_file(fp, crs="EPSG:4326") # have to manually set the projection
basin

## get list of HUC8s within Upper Colorado River Basin
# test = gpd.clip(polys, basin)
# test.plot()

,EXT_ID,EXT_TYP_ID,NAME,geometry
0,9389,5.00,Upper Colorado Region,"POLYGON ((-106.65587 40.51559, -106.65376 40.5..."


In [4]:
## load continental divide shapefile
fp = path_to_data + 'downloads/continental_divide_shapefile/pw312bv3382.shp'
divide = gpd.read_file(fp, crs="ESPG:4326")

## load HU2 shapefile for regions 10, 11, 13, 14
region_lst = [10, 11, 13, 14]
WBD_lst = []
for i, region in enumerate(region_lst):
    fp = path_to_data + 'downloads/WBD_HU2_{0}/Shape/WBDHU2.shp'.format(region)
    WBD = gpd.read_file(fp, crs="ESPG:4326")
    WBD_lst.append(WBD)

## load watershed shapefile
## use geopandas to import the shapefile
fp = path_to_data + 'downloads/CO_HUC8/wbdhu8.shp'
polys = gpd.read_file(fp, crs="epsg:4326") # have to manually set the projection

In [8]:

def calc_fraction(ds1, ds2, varname, thres):
    
    ## total extreme precip
    total_prec = ds2.prec.sum('date').values
    ct_prec = ds2.prec.count('date').values
    wy_prec = ds2.prec.groupby(ds2.water_year).sum(dim="date")
    
    ## get all the dates from the trajectory dataset where criteria is met
    ar_days = ds1.where(ds1[varname] >= float(thres), drop=True).start_date.values
    
    ## select those dates from precip dataset
    ds2 = ds2.sel(date=ar_days)

    ## calculate WY fraction of extreme precip
    wy_ar_prec = ds2.prec.groupby(ds2.water_year).sum(dim="date")
    wy_frac = (wy_ar_prec / wy_prec)*100
    frac_std = float(wy_frac.std(dim='water_year').values / wy_frac.mean(dim='water_year').values)
    
    ## calculate fraction of extreme precipitation related to ARs for each watershed
    ar_prec = ds2.prec.sum('date').values
    frac = (ar_prec/total_prec)*100
    frac_days = (len(ar_days)/ct_prec)*100
    
    return frac, frac_std, ar_prec


def calc_HUC8_AR_contribution(PRISM, HUC8_ID_lst, thres, varname, start_mon, end_mon):
    
    da = xr.DataArray(data=np.array([0, 0, 0, 0, 0, 0]),
                      dims=["ar_scale"],
                      coords=dict(ar_scale=(["ar_scale"], np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0]))))
    
    frac_lst = []
    frac_std_lst = []
    ARscale_lst = []
    ARprec_lst = []
    
    for i, HUC8_ID in enumerate(HUC8_ID_lst):
        # print(i, HUC8_ID)
        # subset the PRISM data to the current HUC8
        tmp = PRISM.sel(HUC8=str(HUC8_ID))
        tmp = tmp.where(tmp.extreme == 1, drop=True)

        ## load final trajectory dataset
        fname = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5_trajectories/combined_extreme_AR/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
        traj = xr.open_dataset(fname)
        traj = select_months_ds(traj, start_mon, end_mon, 'start_date')

        try:
            frac, frac_std, wy_ar_prec = calc_fraction(traj, tmp, varname, thres)

            # get the number of times AR scale for each trajectory
            ARscale = traj.groupby("ar_scale").count().ar

        except ValueError:
            ## no AR days
            frac = 0
            ARscale = da
            wy_ar_prec = 0

        frac_lst.append(frac)
        frac_std_lst.append(frac_std)
        ARscale_lst.append(ARscale)
        ARprec_lst.append(wy_ar_prec)
        
    return frac_lst, frac_std_lst, ARscale_lst, ARprec_lst

In [9]:
start_mon = 5
end_mon = 10

## load PRISM watershed precip dataset
fname = path_to_data + 'preprocessed/PRISM/PRISM_HUC8_CO_sp.nc'
PRISM = xr.open_dataset(fname)
## get select months
PRISM = select_months_ds(PRISM, start_mon, end_mon, 'date')
## add water year to data as coordinate
water_year = (PRISM.date.dt.month >= 10) + PRISM.date.dt.year
PRISM.coords['water_year'] = water_year
HUC8_ID_lst = PRISM.HUC8.values ## get list of HUC8 IDs


thres_lst = [1, 1, 1]
var_lst = ['tARget', 'ar_scale', 'ar']
var_lst2 = ['tARget_std', 'ar_scale_std', 'ar_std']
var_lst3 = ['tARget_prec', 'ar_scale_prec', 'ar_prec']

AR_scale_lst_final = []
for i, (thres, varname, varname2, varname3) in enumerate(zip(thres_lst, var_lst, var_lst2, var_lst3)):
    print('{0}, {1}'.format(varname, thres))
    frac_lst, frac_std_lst, ARscale_lst, ARprec_lst = calc_HUC8_AR_contribution(PRISM, HUC8_ID_lst, thres, varname, start_mon, end_mon)
    ## concat ARscale dataframes
    ds = xr.concat(ARscale_lst, pd.Index(HUC8_ID_lst, name="HUC8"))
    # AR_scale_lst_final.append(ds.sum('HUC8'))
    AR_scale_lst_final.append(ds)
    ## now attach to the geopandas dataframe
    polys[varname] = frac_lst
    polys[varname2] = frac_std_lst
    polys[varname3] = ARprec_lst
    print(polys[varname].max())
    print(polys[varname3].min(), polys[varname3].max())

tARget, 1
47.32814974442958
0 557.9942
ar_scale, 1
57.40295271700979
18.6219 805.1234000000001
ar, 1
32.9554590831594
0 663.7571


In [10]:
HUC8_ID_lst = [14050001, ## upper yampa
               # 14050005, ## upper white
               # 14050002, ## lower yampa
               # 14080104, ## animas (San Juans),
               # 14030002, ## upper dolores
               13010001, ## rio grande headwaters
               # 11020002, ## arkansas - Pueblo Reservoir
               # 10190005 ## St. Vrain (Boulder)
               # 14030005, ## 'Upper Colorado-Kane Springs'
               # 14010001, ## Colorado Headwaters
               10190002, ## 'Upper South Platte'
               11020001 ## Arkansas Headwaters
              ]
idx = (polys.HUC8 == str(HUC8_ID_lst[0])) | (polys.HUC8 == str(HUC8_ID_lst[1])) | (polys.HUC8 == str(HUC8_ID_lst[2])) | (polys.HUC8 == str(HUC8_ID_lst[3]))
tmp =  polys[idx]
tmp

,OBJECTID,TNMID,MetaSource,SourceData,SourceOrig,SourceFeat,LoadDate,GNIS_ID,AreaAcres,AreaSqKm,...,geometry,tARget,tARget_std,tARget_prec,ar_scale,ar_scale_std,ar_scale_prec,ar,ar_std,ar_prec
20,21,{E174065A-2AA4-49D1-B443-F550A60C869D},None,None,None,None,2016-07-27,0,"1,676,731.25","6,785.50",...,"POLYGON ((-106.77828 40.90596, -106.77765 40.9...",15.66,0.56,186.51850000000002,25.63,0.55,305.2108,15.79,0.57,187.9941
49,50,{19D18F78-BA7C-4A1E-A1F6-DA996B233880},None,None,None,None,2017-04-17,0,"1,961,578.79","7,938.23",...,"POLYGON ((-106.21421 39.38309, -106.21368 39.3...",6.39,0.58,84.0356,8.32,0.45,109.4967,6.71,0.45,88.2925
86,87,{A8098AC6-DFD5-48AA-A352-9057E09913D8},None,None,None,None,2017-09-19,0,"883,590.41","3,575.77",...,"POLYGON ((-106.96559 37.97355, -106.96542 37.9...",47.33,0.49,557.9942,52.85,0.48,623.0584,32.96,0.61,388.5416
87,88,{AAFC1326-6A7C-49C0-BC17-9573AE42BC21},None,None,None,None,2016-10-11,0,"1,183,831.16","4,790.80",...,"POLYGON ((-105.04506 39.76261, -105.04449 39.7...",1.07,0.00,17.9928,2.94,0.58,49.3565,1.07,0.00,17.9928


In [ ]:
polys.sort_values(by=['ar_scale'])

In [ ]:
polys['ar_std'].values.min()

In [ ]:
AR_scale_lst_final[2].sum('HUC8')

In [ ]:
for i, HUC8 in enumerate(HUC8_ID_lst):
    print(AR_scale_lst_final[2].sel(HUC8=str(HUC8)))

In [ ]:
# Set up projection
datacrs = ccrs.PlateCarree()  ## the projection the data is in
mapcrs = ccrs.PlateCarree() ## the projection you want your map displayed in

# Set tick/grid locations
ext1 = [-111., -100., 35.5, 42.5] # extent of CO
dx = np.arange(ext1[0],ext1[1]+2,2)
dy = np.arange(ext1[2]-.5,ext1[3]+1,1)

# # make a colormap that has land and ocean clearly delineated and of the
# # same length (256 + 256)
# colors_undersea = plt.cm.terrain(np.linspace(0, 0.17, 256))
# colors_land = plt.cm.terrain(np.linspace(0.25, 1, 256))
# all_colors = np.vstack((colors_undersea, colors_land))
# terrain_map = mcolors.LinearSegmentedColormap.from_list(
#     'terrain_map', all_colors)
# # make the norm:  Note the center is offset so that the land has more
# # dynamic range:
# divnorm = mcolors.TwoSlopeNorm(vmin=-0.25, vcenter=1, vmax=3000)


wc_rgb = (86./255., 105./255., 166./255.) # color for water features
wc_cmyk = (48., 37., 0., 35.)

# list of letters to append to titles
letter_lst = list(map(chr, range(97, 123)))


In [ ]:
# Create figure
fig = plt.figure(figsize=(8.5, 11))
fig.dpi = 300
fname = path_to_figs + 'choropleth_map_portrait_warm_ssn'
fmt = 'png'

nrows = 5
ncols = 2

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1, 1, 1, 0.05, 0.05], width_ratios = [1, 1], wspace=0.02, hspace=0.02)
## use gs[rows index, columns index] to access grids

# Add color bar axis
cbax = plt.subplot(gs[-1,0]) # colorbar axis
lbl_lst = ['tARget v4', 'AR scale', 'Rutz AR']
var_lst = ['tARget', 'ar_scale', 'ar']
row_idx = [0, 1, 2]
b_lons = [False, False, True]

for k, varname in enumerate(var_lst):
    print(k, row_idx[k], varname)
    ## Add axis for plot
    ax = fig.add_subplot(gs[row_idx[k],0], projection=mapcrs)
    ax = draw_basemap(ax, extent=ext1, xticks=dx, yticks=dy,left_lats=True, right_lats=False, bottom_lons=b_lons[k], mask_ocean=False, coastline=False)

    ## topo with gray shading
    # cs = plot_terrain(ax, ext1)

    # add choropleth watershed fraction
    cbarticks = [10, 20, 30, 40, 50, 60, 70]
    lgnd_kwds={"label": "Fraction of top-decile AR precipitation (%)", "orientation": "horizontal", "ticks": cbarticks}
    cmap, norm, bnds = ccmap.cmap_segmented(cmo.rain, np.arange(0, 90, 10))
    cf = polys.plot(ax=ax, column=varname, cmap=cmap, vmin=0, vmax=80, norm=norm, alpha=0.8, legend=True, cax=cbax, legend_kwds=lgnd_kwds)
    polys.plot(ax=ax, edgecolor='grey', color='None', linewidth=0.5, zorder=98)

    ax.add_feature(cfeature.STATES, edgecolor='0.4', linewidth=0.8, zorder=199)

    ## add in four focus watersheds
    tmp.plot(ax=ax, edgecolor='white', color='None', linewidth=1., zorder=99)
    # basin.plot(ax=ax, edgecolor='white', color='None', zorder=99)

    ## add in region watershed shape - first time full opacity, second low opacity
    opac_lst = [1., 0.9]
    zord_lst = [98, 100]
    lw_lst = [0.75, 0.3]
    for (opac, zord, lw) in zip(opac_lst, zord_lst, lw_lst):
        for j, WBD in enumerate(WBD_lst):
            WBD.plot(ax=ax, edgecolor='k', color='None', linewidth=lw, zorder=zord, alpha=opac)

    ## add continental divide
    # divide.plot(ax=ax, color='tab:blue', zorder=99)

    # ax.set_title(lbl_lst[k], loc='left', fontsize=14)
    ax.text(-0.16, 0.5, lbl_lst[k], va='bottom', ha='center',
                    rotation='vertical', rotation_mode='anchor', fontsize=13,
                    transform=ax.transAxes)

    ## add a, b, c labels
    titlestring = '({0})'.format(letter_lst[k])
    ax.text(0.029, 0.973, titlestring, ha='left', va='top',
            transform=ax.transAxes, fontsize=12., backgroundcolor='white', zorder=101)

cbax = plt.subplot(gs[-1,1]) # colorbar axis
row_idx = [0, 1, 2]
b_lons = [False, False, True]

for k, varname in enumerate(var_lst2):
    ## Add axis for plot
    ax = fig.add_subplot(gs[row_idx[k],1], projection=mapcrs)
    ax = draw_basemap(ax, extent=ext1, xticks=dx, yticks=dy,left_lats=False, bottom_lons=b_lons[k], right_lats=False, mask_ocean=False, coastline=False)

    # add choropleth watershed fraction
    cbarticks = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
    lgnd_kwds={"label": "Coefficient of Variation (-)", "orientation": "horizontal", "ticks": cbarticks}
    cmap, norm, bnds = ccmap.cmap_segmented(cmo.rain, np.arange(0, 1.3, .1))
    cf = polys.plot(ax=ax, column=varname, cmap=cmap, vmin=0, vmax=30, norm=norm, alpha=0.8, legend=True, cax=cbax, legend_kwds=lgnd_kwds)
    polys.plot(ax=ax, edgecolor='grey', color='None', linewidth=0.5, zorder=97)

    ax.add_feature(cfeature.STATES, edgecolor='0.4', linewidth=0.8, zorder=199)

    ## add in four focus watersheds
    tmp.plot(ax=ax, edgecolor='white', color='None', linewidth=1., zorder=99)
    # basin.plot(ax=ax, edgecolor='white', color='None', zorder=99)

    ## add in region watershed shape - first time full opacity, second low opacity
    opac_lst = [1., 0.9]
    zord_lst = [98, 100]
    lw_lst = [0.75, 0.3]
    for (opac, zord, lw) in zip(opac_lst, zord_lst, lw_lst):
        for j, WBD in enumerate(WBD_lst):
            WBD.plot(ax=ax, edgecolor='k', color='None', linewidth=lw, zorder=zord, alpha=opac)


    ## add a, b, c labels
    titlestring = '({0})'.format(letter_lst[k+3])
    ax.text(0.029, 0.973, titlestring, ha='left', va='top',
            transform=ax.transAxes, fontsize=12., backgroundcolor='white', zorder=101)
        
fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()